In [1]:
import json
path='/root/projects/mmcot/gqa/train_all_questions/train_all_questions_0.json'
path_bal='/root/projects/mmcot/gqa/val_balanced_questions.json'
path_bal='/root/projects/mmcot/gqa/testdev_balanced_questions.json'
# with open(path,'r') as f:
#     data=json.load(f)
with open(path_bal,'r') as f:
    data_bal=json.load(f)

cnt=0
infos=[]

for j,(key, value) in enumerate(data_bal.items()):

    # if check_nouns(value['question']):
    if j%6==0 and cnt<600: 
        cnt+=1
        continue
    if j%6==0 and cnt<800:
        info={}
        cnt+=1

        
        info['question']=value['question']
        info['answer']=value['answer']
        info['imageId']=value['imageId']
        infos.append(info)
# import pickle
# with open('/root/projects/code_lihan/Open_vocab_det/QAI.pkl','wb') as f:
#     pickle.dump(infos,f)


In [2]:
import pickle 
# with open("/root/projects/code_lihan/experiment_questions_data/QAI.pkl",'rb') as f:
#     data=pickle.load(f)
# print(len(data))
questions=[]
imageIds=[]
answers=[]
for i in range(200):
    d=infos[i]
    questions.append(d['question'])
    imageIds.append(d['imageId']) 
    answers.append(d['answer']) 


Generate nodes according to the question.

In [5]:
import sys
sys.path.append('/root/projects/code_lihan/')
import graph_generation
import importlib
importlib.reload(graph_generation)
from graph_generation import entity_generation
nodes,_=entity_generation(questions)
# tmpN=[]
# tmpQ=[]
# tmpI=[]
# tmpA=[]
# for i in range(len(nodes)):
#     node=nodes[i]
#     question=questions[i]
#     imageId=imageIds[i]
#     answer=answers[i]
#     if len(node)>=0:
#         tmpN.append(node)
#         tmpQ.append(question)
#         tmpI.append(imageId)
#         tmpA.append(answer)
# nodes=tmpN
# questions=tmpQ
# answers=tmpA
# imageIds=tmpI
# # for i in range(len(nodes)):
# for i in range(20):
#     node=nodes[i]
#     question=questions[i]
#     imageId=imageIds[i]
#     answer=answers[i]
#     print(node,question,imageId,answer)

========== Entity and Question ===========


0
100


In [6]:
num=600
with open(f'{num}-{num+200}_Q.txt','w') as f:
    for q in questions:
        f.write(q+'\n')
# with open('MoreThanTwoGoodImg.txt','w') as f:
with open(f'{num}-{num+200}_Img.txt','w') as f:
    for q in imageIds:
        f.write(q+'\n')
with open(f'{num}-{num+200}_A.txt','w') as f:
    for a in answers:
        f.write(a+'\n')


In [7]:
import json
with open(f'{num}-{num+200}_nodes.json',"w") as f:
    json.dump(nodes,f)

In [3]:
# import json
num=600
with open(f'{num}-{num+200}_nodes.json', 'r') as file:
    nodes = json.load(file)
nodes

[['person', 'playing'],
 ['umbrella'],
 ['bookcase', 'desk'],
 ['furniture', 'spoon', 'her'],
 ['wristband'],
 ['basket', 'train'],
 ['animal', 'grass'],
 ['person', 'woman'],
 ['furniture'],
 ['vehicle'],
 ['curtain'],
 ['stairs'],
 ['woman', 'bag'],
 ['beans'],
 ['lamps', 'mirrors'],
 ['device', 'speaker'],
 ['sidewalk', 'concrete'],
 ['person', 'crouching person'],
 ['jeans', 'sneakers'],
 ['animal'],
 ['building', 'sign'],
 ['old woman', 'man', 'giraffe'],
 ['man'],
 ['pants'],
 ['mountains'],
 ['helmet', 'bench'],
 ['person', 'bending', 'resting', 'reading'],
 ['red bricks'],
 ['bottle cap'],
 ['drink', 'cup'],
 ['telephone pole', 'train', 'bus'],
 ['curtains'],
 ['appliance', 'knives'],
 ['lamp', 'metal'],
 ['motorcycle', 'person'],
 ['ground'],
 ['soccer player', 'goal', 'ball'],
 ['fast food', 'mug'],
 ['vegetable', 'stove top'],
 ['appliance', 'oil', 'wine'],
 ['pasture', 'horse'],
 ['towel'],
 ['glass', 'glass', 'object'],
 ['couch', 'cloth'],
 ['skiing'],
 ['person', 'shirt'

In [3]:
with open(f'{num}-{num+200}_Q.txt','r') as f:
# with open('MoreThanTwoGoodQ.txt','r') as f:
    questions=[]
    for line in f:
        questions.append(line[:-1])
print(len(questions))
# with open('MoreThanTwoGoodImg.txt','r') as f:
with open(f'{num}-{num+200}_Img.txt','r') as f:
    imageIds=[]
    for line in f:
        imageIds.append(line[:-1])
print(len(imageIds))


0
200


In [15]:
import sys
sys.path.append('/root/projects/code_lihan/')
sys.path.append('/root/projects/code_lihan/GroundingDINO')
import importlib
import objects_graph
import groundingdino.util.inference 
importlib.reload(groundingdino.util.inference)
importlib.reload(objects_graph)
from objects_graph import ObjModel
import os 
# config_file = '/root/projects/mmcot/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py'  # change the path of the model config file

# grounded_checkpoint = '/root/projects/mmcot/GroundingDINO/weights/groundingdino_swint_ogc.pth'  # change the path of the model
config_file = '/root/projects/mmcot/GroundingDINO/groundingdino/config/GroundingDINO_SwinB_cfg.py'  # change the path of the model config file
# # ram_checkpoint = '/root/autodl-tmp/eva_weights/ram_plus_swin_large_14m.pth'  # change the path of the model
grounded_checkpoint = '/root/projects/mmcot/GroundingDINO/weights/groundingdino_swinb_cogcoor.pth'  # change the path of the model
model = ObjModel(grounded_checkpoint, config_file)
obj_dicts=[]
axis_dicts=[]
obj_not_founds=[]
obj_filt_dicts=[]
# path_out='./test_morethan2'

os.makedirs(f'./test_{num}-{num+200}', exist_ok=True)
image_processed_Ids=[]
for i in range(len(nodes)):
# 调用方法
    # if i < 88: continue
    objlist = nodes[i]

    # if img_name in os.listdir(path_out):
    #     img_id_out=imageIds[i]
    # else:
    img_id_out=str(i)+'_'+imageIds[i]
    
    print("img_name: ",img_id_out)
    image_processed_Ids.append(img_id_out)
    obj_dict,obj_not_found ,obj_filt_dict= model.find_obj( objlist, img_name, img_id_out, path,path_out,need_obj_boxes=False)
    print("obj_dict",obj_dict)

    dict_ori=obj_dict
    obj_dict={}
    axis_dict={}
    for key,val in dict_ori.items():
        num_=[]
        for key_obj,val_obj in val.items():
            num_.append(val_obj['number'])
            axis_dict[val_obj['number']]=val_obj['bbox']
        obj_dict[key]=num_
    obj_dicts.append(obj_dict)
    axis_dicts.append(axis_dict)
    obj_not_founds.append(obj_not_found)
    obj_filt_dicts.append(obj_filt_dict)
    # if i==20: break
    # break
import json
with open(f'obj_dicts_{num}-{num+200}.json','w') as f:
    json.dump(obj_dicts,f)
with open(f'axis_dicts_{num}-{num+200}.json','w') as f:
    json.dump(axis_dicts,f)
with open(f'obj_not_founds_{num}-{num+200}.json','w') as f:
    json.dump(obj_not_founds,f)
with open(f'obj_filt_dicts_{num}-{num+200}.json','w') as f:
    json.dump(obj_filt_dicts,f)
with open(f'image_processed_Ids_{num}-{num+200}.txt','w') as f:
    for imageId in image_processed_Ids:
        f.write(str(imageId)+'\n')


final text_encoder_type: bert-base-uncased


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

_IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight', 'bert.embeddings.position_ids'])
==========loading model successfully=============
img_name:  0_n314171


boxes_filt-- tensor([[  0.0000, 386.6590, 320.5542, 427.0000]])
pred_phrase-- ['red table(0.56)']
boxes_filt-- tensor([[176.0974, 226.0165, 217.2541, 263.7041],
        [134.9921, 226.7801, 176.3076, 294.6703],
        [167.3370, 256.6122, 243.7745, 283.2394],
        [ 34.0076, 415.3821,  70.9305, 427.0000],
        [208.4385, 225.2156, 237.2335, 250.4910],
        [210.6879, 218.0341, 256.4754, 244.8178],
        [ 63.7150, 417.0414, 105.2563, 427.0000],
        [240.0275, 226.0587, 279.6311, 246.0557]])
pred_phrase-- ['bottle(0.34)', 'bottle(0.34)', 'bottle(0.34)', 'bottle(0.32)', 'bottle(0.32)', 'bottle(0.31)', 'bottle(0.30)', 'bottle(0.27)']
after sort [(321, 0, 0), (160, 176, 1), (208, 134, 2), (256, 167, 3), (369, 34, 4), (112, 208, 5), (82, 210, 6), (417, 63, 7), (208, 240, 8)]
Adjusted collect_boxes_filt [tensor([[  0.0000, 321.0000, 320.5542, 427.0000]]), tensor([[176.0974, 160.0000, 217.2541, 263.7041],
        [134.9921, 208.0000, 176.3076, 294.6703],
        [167.3370, 256

Start running~

In [1]:
num=600
import json
with open(f'obj_dicts_{num}-{num+200}.json','r') as f:
    obj_dicts=json.load(f)
with open(f'axis_dicts_{num}-{num+200}.json','r') as f:
    axis_dicts=json.load(f)
with open(f'obj_not_founds_{num}-{num+200}.json','r') as f:
    obj_not_founds=json.load(f)
with open(f'{num}-{num+200}_Q.txt','r') as f:
    questions=[]
    for line in f:
        questions.append(line[:-1])
print(len(questions))
with open(f'image_processed_Ids_{num}-{num+200}.txt','r') as f:
    imageIds=[]
    for line in f:
        imageIds.append(line[:-1])
print(len(imageIds))
with open(f'{num}-{num+200}_A.txt','r') as f:
    answers=[]
    for line in f:
        answers.append(line[:-1])
print(len(answers))
with open(f'obj_filt_dicts_{num}-{num+200}.json','r') as f:
    obj_filt_dicts=json.load(f)

200
200
200


In [2]:
import sys
sys.path.append('/root/projects/code_lihan/')
import importlib
import json
import tools
import traverse_node_edge
import graph_generation
import langgraph_process
# import gpt4_img
import gpt4_img_qwen
import re
import string
# importlib.reload(gpt4_img)
importlib.reload(gpt4_img_qwen)
importlib.reload(traverse_node_edge)
importlib.reload(graph_generation)
importlib.reload(tools)
importlib.reload(langgraph_process)
import os
os.environ["DASHSCOPE_API_KEY"]= 'sk-ac7aca0206ae4da9a517628e5fa2170f'


# questions=['Are there napkins under the utensil to the left of the rice?']
img_dirs=[]
for imageId in imageIds:
    img_dirs.append(f'/root/projects/code_lihan/experiment_questions_data/test_{num}-{num+200}/'+imageId+'.jpg')
# obj_dicts=[{'napkins': [1], 'utensil': [2, 3], 'rice': [4]}]
cnt=0
# collection=[]
for imageId,question,img_dir,label,obj_dict,axis_dict,obj_not_found,obj_filt_dict in zip(imageIds,questions,img_dirs,answers,obj_dicts,axis_dicts,obj_not_founds,obj_filt_dicts):
    if cnt<30:
        cnt+=1
        continue 
    if cnt==120:
        break
    question=question[0]+question[1:].lower()
    question_words=question[:-1].split(' ')
    # if 'Who' in question_words :
    #     # cnt+=1
    #     continue
    # if 'big' not in question_words :
    #     # cnt+=1
    #     continue
    question = re.sub(r'walking\b', 'standing', question)
    question = re.sub(r'walk\b', 'stand', question)
    question = re.sub(r'Where\b', 'What place', question)
    question = re.sub(r'name\b', 'type', question)
    
    cnt+=1
    to_remove=[]
    for k,v in obj_filt_dict.items():
        if k not in obj_dict:
            to_remove.append(k)
    for k in to_remove:
        del obj_filt_dict[k]
    # if 'left' not in question and 'right' not in question:
    #     continue
    print("\n")
    print(f"The {cnt} sample:")
    print(imageId,question,label,obj_dict)
    Questions=[question]
    Img_dirs=[img_dir]
    Obj_dicts=[obj_dict]
    Axis_dicts=[axis_dict]
    Obj_not_found=[obj_not_founds]
    Obj_filt_dict=[obj_filt_dict]
    coll_dict={}

    entity_list=[]
    for obj_dict in Obj_dicts:
        entity_list.append(list(obj_dict.keys()))
    # if len(entity_list[0])>0:
    #     continue
    # print('obj_filt_dict',obj_filt_dict)
    # print('obj_dict',obj_dict)
    # payload = create_payload_final_ans( img_name, question,'','')
    # response = query_openai(payload)
    # # print(question)
    # logit=response['choices'][0]['message']['content']
    # print(response['choices'][0]['message']['content'])
    # print('answer',label)
    image_ori_path='/root/projects/mmcot/gqa/images'
    logit=traverse_node_edge.main(Questions,Img_dirs,Obj_dicts,Axis_dicts,entity_list,Obj_not_found,Obj_filt_dict,image_ori_path)

    # logit=graph_generation.main(Questions,entity_list)
    coll_dict['id']=cnt
    coll_dict['Label']=label
    coll_dict['Logit']=logit
    coll_dict['Q']=question
    coll_dict['img_name']=imageId
    collection.append(coll_dict)
    if cnt==1:
        print(collection)
    # if cnt%80==0:
    #     with open("all_1000-1200_collection.json","a") as f:
    #         json.dump(collection,f)
    # if cnt==6:
    #     break
# with open('obj_filt_dicts_who.json','r') as f:
#     obj_filt_dicts=json.load(f)
# with open('obj_dicts_who.json','r') as f:
#     obj_dicts=json.load(f)

    # if cnt==18:
        # break
# with open('obj_dicts_100-200.json','r') as f:
# with open('obj_dicts_0-100.json','r') as f:
#     obj_dicts=json.load(f)

USER_AGENT environment variable not set, consider setting it to identify your requests.




The 31 sample:
30_n518912 Are the curtains white and short? no {'curtains': [1]}
all_nodes [['curtains']]
========== Node question Generation ===========
False
all_node ['curtains']
node_question {"curtains":["Are the curtains white?","Are the curtains short?"]}
node_question,posi_q {'curtains': ['Are the curtains white?', 'Are the curtains short?']} {'curtains': []}
========== Dict Generation ===========
['Are the curtains white and short?']
extracted_lists [[]]
=========generate edge============
No edge_question for this question.
all_edgequestions edge [['No edge_question']]
all_relationship_dict:  [{}]
combined_relation {}
==========Generated graph==========
[{'curtains': {'node_question': ['Are the curtains white?', 'Are the curtains short?']}}]
Main_node: curtains
  Main_node Question: Are the curtains white?

obj_dict to update the question {'curtains': [1]}
updated_question  Are the curtains [1] white?
label_sentence In the image, there are one curtains marked by 1.
{'agent':

KeyboardInterrupt: 

TypeError: Conversation.call() missing 1 required positional argument: 'self'


In [12]:
import sys
sys.path.append('/root/projects/code_lihan/')
import importlib
import json
import tools
import traverse_node_edge
import graph_generation
import langgraph_process
import gpt4_img
import re
import string
importlib.reload(gpt4_img)
importlib.reload(traverse_node_edge)
importlib.reload(graph_generation)
importlib.reload(tools)
importlib.reload(langgraph_process)



# questions=['Are there napkins under the utensil to the left of the rice?']
img_names=[]
for imageId in imageIds:
    img_names.append(f'/root/projects/code_lihan/experiment_questions_data/test_{num}-{num+200}/'+imageId+'.jpg')
# obj_dicts=[{'napkins': [1], 'utensil': [2, 3], 'rice': [4]}]
cnt=0
collection=[]
for imageId,question,img_name,label,obj_dict,axis_dict,obj_not_found,obj_filt_dict in zip(imageIds,questions,img_names,answers,obj_dicts,axis_dicts,obj_not_founds,obj_filt_dicts):
    if cnt<43:
        cnt+=1
        continue 
    question=question[0]+question[1:].lower()
    question_words=question[:-1].split(' ')
    # if 'Who' in question_words :
    #     # cnt+=1
    #     continue
    # if 'big' not in question_words :
    #     # cnt+=1
    #     continue
    question = re.sub(r'walking\b', 'standing', question)
    question = re.sub(r'walk\b', 'stand', question)
    question = re.sub(r'Where\b', 'What place', question)
    question = re.sub(r'name\b', 'type', question)
    
    cnt+=1
    to_remove=[]
    for k,v in obj_filt_dict.items():
        if k not in obj_dict:
            to_remove.append(k)
    for k in to_remove:
        del obj_filt_dict[k]
    # if 'left' not in question and 'right' not in question:
    #     continue
    print("\n")
    print(f"The {cnt} sample:")
    print(imageId,question,label,obj_dict)
    Questions=[question]
    Img_names=[img_name]
    Obj_dicts=[obj_dict]
    Axis_dicts=[axis_dict]
    Obj_not_found=[obj_not_founds]
    Obj_filt_dict=[obj_filt_dict]
    coll_dict={}

    entity_list=[]
    for obj_dict in Obj_dicts:
        entity_list.append(list(obj_dict.keys()))
    # print('obj_filt_dict',obj_filt_dict)
    # print('obj_dict',obj_dict)
    # payload = create_payload_final_ans( img_name, question,'','')
    # response = query_openai(payload)
    # # print(question)
    # logit=response['choices'][0]['message']['content']
    # print(response['choices'][0]['message']['content'])
    # print('answer',label)
    logit=traverse_node_edge.main(Questions,Img_names,Obj_dicts,Axis_dicts,entity_list,Obj_not_found,Obj_filt_dict)
    # logit=graph_generation.main(Questions,entity_list)
    coll_dict['id']=cnt
    coll_dict['Label']=label
    coll_dict['Logit']=logit
    coll_dict['Q']=question
    coll_dict['img_name']=img_name
    collection.append(coll_dict)
    if cnt==1:
        print(collection)
    # if cnt%80==0:
    #     with open("all_1000-1200_collection.json","a") as f:
    #         json.dump(collection,f)
    break
# with open('obj_filt_dicts_who.json','r') as f:
#     obj_filt_dicts=json.load(f)
# with open('obj_dicts_who.json','r') as f:
#     obj_dicts=json.load(f)

    # if cnt==18:
        # break
# with open('obj_dicts_100-200.json','r') as f:
# with open('obj_dicts_0-100.json','r') as f:
#     obj_dicts=json.load(f)

In [32]:
import json
with open("qwen_test_600-720collection.json","w") as f:
    json.dump(collection,f)

In [33]:
import json
import re
with open("qwen_test_600-720collection.json","rb") as f:
    collections=json.load(f)
cnt=0
nums=[]
cnt1=0
for c in collections:
    # if 'what' in c['Q'] or 'What' in c['Q']: continue
    # cnt1+=1
    if c['Logit']==c['Label']:cnt+=1
    else:
        if bool(re.search(r'\d', c['Logit'])): nums.append(c['id'])
        # print('Logit: ',c['Logit'])
        # print('Label: ',c['Label'])
print(cnt)
print(cnt1)

31
0


In [3]:
import json
import re
with open("test_0-200_collection.json","rb") as f:
    collections=json.load(f)
cnt=0
nums=[]
cnt1=0
for c in collections:
    # if 'what' in c['Q'] or 'What' in c['Q']: 
    #     print(c['Q'])
    #     continue
    cnt1+=1
    if c['Logit']==c['Label']:cnt+=1
    else:
        if bool(re.search(r'\d', c['Logit'])): nums.append(c['id'])
        # print('Logit: ',c['Logit'])
        # print('Label: ',c['Label'])
print(cnt)
print(cnt1)

101
189


In [7]:
import sys
sys.path.append('/root/projects/code_lihan/')
import importlib
import gpt4_img
importlib.reload(gpt4_img)

from gpt4_img import create_payload_normal,query_openai,create_payload_final_ans,create_payload_edge,create_payload_node
# for i in range(len(imageIds)):
    # imageId=str(imageIds[i])
    # question=questions[i]
    # answer=answers[i]
# The white lamp is marked as 1, and the white curtain is marked as 2.

payload = create_payload_final_ans( "/root/projects/code_lihan/experiment_questions_data/test_300-600_exWho/125_2363769.jpg", "What is the size of the book above the teddy bear, compared with a usual book? Answer with large or small.",'','')
response = query_openai(payload)
# print(question)
print(response['choices'][0]['message']['content'])
# print(answer)

image_dir /root/projects/code_lihan/experiment_questions_data/test_300-600_exWho/125_2363769.jpg
What is the size of the book above the teddy bear, compared with a usual book? Answer with large or small in the image?
Contexto_to_gpt4: My VQA question is: What is the size of the book above the teddy bear, compared with a usual book? Answer with large or small in the image?
Small


In [13]:
# for imageId,question,img_name,label,obj_dict,axis_dict,obj_not_found,obj_filt_dict in zip(imageIds,questions,img_names,answers,obj_dicts,axis_dicts,obj_not_founds,obj_filt_dicts):
#     if cnt<0: 
#         cnt+=1
#         continue 
#     question_words=question[:-1].split(' ')
#     if 'there' not in question_words and 'see' not in question_words:
#         cnt+=1
#         continue
#     # if cnt==34: 
#     #     cnt+=1
#     #     continue
#     cnt+=1
#     imageId=imageId.split('_')[1]
#     print(f"/root/projects/mmcot/gqa/images/{imageId}.jpg")
#     payload = create_payload_final_ans( f"/root/projects/mmcot/gqa/images/{imageId}.jpg", question,'','')
#     response = query_openai(payload)
#     # print(question)
#     print(response['choices'][0]['message']['content'])
#     print('answer',label)
